In [3]:
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('tirthadagr8/custom-mbart-large-50')
# tokenizer=AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B')

In [3]:
import os

os.environ['NEPTUNE_API_TOKEN'] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2ZGUwMDYyOC04NmE0LTQyM2UtOTVjNi0wZjQ3ZGU2ZjM4M2IifQ=="
os.environ['NEPTUNE_PROJECT'] = 'tirthadagr8/model-feed'
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
config=GPT2Config(vocab_size=len(tokenizer),bos_token_id=tokenizer.bos_token_id,eos_token_id=tokenizer.eos_token_id,n_embd=384,n_layer=12,n_head=8)
no_of_parameters=config.vocab_size*config.n_embd+config.n_layer*config.n_embd+config.n_layer*(4*config.n_embd*config.n_embd+4*config.n_embd+2*config.n_embd*4*config.n_embd+9*config.n_embd)+2*config.n_embd
size_of_model=no_of_parameters/(1.6*100000000)
print(f'Number of parameters would be:{no_of_parameters} the size would be:{size_of_model}')

Number of parameters would be:118813824 the size would be:0.7425864


In [5]:
model = GPT2LMHeadModel(config)

In [6]:
with open('C:/Users/tirth/Desktop/english_corpus.txt','r') as f:
    text=f.readline()
chunk_size = 128  # You can adjust this value
array_of_strings = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [7]:
from datasets import load_dataset,Dataset
from transformers import LlamaTokenizer

# Load text data
dataset = Dataset.from_dict({'text':array_of_strings})
split_dataset = dataset.train_test_split(test_size=0.1)  # 80% train, 20% test
# Initialize the tokenizer
# tokenizer = LlamaTokenizer.from_pretrained('huggingface/llama-tokenizer')  # or use a compatible LLaMA tokenizer

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_dataset = split_dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

Map:   0%|          | 0/66779 [00:00<?, ? examples/s]

Map:   0%|          | 0/7420 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to False since we’re not using masked language modeling
)

In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 66779
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 7420
    })
})

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-scratch",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01
)

In [11]:
from transformers import Trainer
from transformers.integrations import NeptuneCallback
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=test_dataset,
    # callbacks=[NeptuneCallback(api_token=os.getenv('NEPTUNE_API_TOKEN'), project=os.getenv('NEPTUNE_PROJECT'))]
)

In [12]:
trainer.train()

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/tirthadagr8/model-feed/e/MOD-57


c:\Users\tirth\AppData\Local\Programs\Python\Python312\Lib\site-packages\neptune\internal\utils\git.py:71: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


  0%|          | 0/50085 [00:00<?, ?it/s]

{'loss': 8.764, 'grad_norm': 4.845093727111816, 'learning_rate': 4.9900169711490466e-05, 'epoch': 0.01}
{'loss': 5.0168, 'grad_norm': 2.814631938934326, 'learning_rate': 4.980033942298094e-05, 'epoch': 0.01}
{'loss': 2.986, 'grad_norm': 1.561206340789795, 'learning_rate': 4.97005091344714e-05, 'epoch': 0.02}
{'loss': 2.6854, 'grad_norm': 1.8047361373901367, 'learning_rate': 4.9600678845961865e-05, 'epoch': 0.02}
{'loss': 2.625, 'grad_norm': 1.6163281202316284, 'learning_rate': 4.9500848557452335e-05, 'epoch': 0.03}
{'loss': 2.5847, 'grad_norm': 1.3443793058395386, 'learning_rate': 4.94010182689428e-05, 'epoch': 0.04}
{'loss': 2.5664, 'grad_norm': 1.3705471754074097, 'learning_rate': 4.930118798043327e-05, 'epoch': 0.04}
{'loss': 2.5504, 'grad_norm': 1.627346158027649, 'learning_rate': 4.9201357691923734e-05, 'epoch': 0.05}
{'loss': 2.5263, 'grad_norm': 1.4689449071884155, 'learning_rate': 4.91015274034142e-05, 'epoch': 0.05}
{'loss': 2.5377, 'grad_norm': 1.2699191570281982, 'learning_r

KeyboardInterrupt: 